In [63]:
import pandas as pd
import numpy as np
import pickle
import time
from scipy.sparse import csr_matrix
from scipy.stats import kendalltau, spearmanr
from sklearn.linear_model import LogisticRegression, LinearRegression
from itertools import combinations

# Пункт 1: данные

*Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:*
- *взять в тренировочный набор турниры с dateStart из 2019 года;* 
- *в тестовый — турниры с dateStart из 2020 года.*


Данные состоят из трех файлов:
- players.pkl
- tournaments.pkl
- results.pkl

Посмотрим на каждый из них

## players

In [2]:
with open('chgk/players.pkl', 'rb') as file:
    players_df = pd.DataFrame.from_dict(pickle.load(file), orient = 'index')

print('Размер: ', players_df.shape)        
players_df.head()

Размер:  (204063, 4)


,id,name,patronymic,surname
1,1,Алексей,None,Абабилов
10,10,Игорь,,Абалов
11,11,Наталья,Юрьевна,Абалымова
12,12,Артур,Евгеньевич,Абальян
13,13,Эрик,Евгеньевич,Абальян


## tournaments

In [3]:
with open('chgk/tournaments.pkl', 'rb') as file:
    tournaments_df = pd.DataFrame.from_dict(pickle.load(file), orient = 'index')

print('Размер: ', tournaments_df.shape)    
tournaments_df.head()

Размер:  (5528, 9)


,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
1,1,Чемпионат Южного Кавказа,2003-07-25T00:00:00+04:00,2003-07-27T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
2,2,Летние зори,2003-08-09T00:00:00+04:00,2003-08-09T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
3,3,Турнир в Ижевске,2003-11-22T00:00:00+03:00,2003-11-24T00:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
4,4,Чемпионат Украины. Переходной этап,2003-10-11T00:00:00+04:00,2003-10-12T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
5,5,Бостонское чаепитие,2003-10-10T00:00:00+04:00,2003-10-13T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None


## results

In [4]:
with open('chgk/results.pkl', 'rb') as file:
    results_df = pd.DataFrame.from_dict(pickle.load(file), orient = 'index')

print('Размер: ', results_df.shape)
results_df.head(2)

Размер:  (5528, 2064)


,0,1,2,3,4,5,6,7,8,9,...,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063
1,"{'team': {'id': 242, 'name': 'Команда Азимова'...","{'team': {'id': 640, 'name': 'Перезагрузка', '...","{'team': {'id': 245, 'name': 'Айастан', 'town'...","{'team': {'id': 299, 'name': 'Команда Гусейнов...","{'team': {'id': 1189, 'name': 'Грааль', 'town'...","{'team': {'id': 251, 'name': 'Пентиум', 'town'...","{'team': {'id': 268, 'name': 'Команда Алиева',...","{'team': {'id': 307, 'name': 'Третий путь', 't...","{'team': {'id': 543, 'name': 'Команда Мусаева'...","{'team': {'id': 41, 'name': 'Фантазия', 'town'...",...,None,None,None,None,None,None,None,None,None,None
2,"{'team': {'id': 312, 'name': 'Социал-демократы...","{'team': {'id': 510, 'name': 'Импульс', 'town'...","{'team': {'id': 195, 'name': 'Реал-Мордовия', ...","{'team': {'id': 264, 'name': 'Джокер', 'town':...","{'team': {'id': 240, 'name': 'Бегемот', 'town'...","{'team': {'id': 689, 'name': 'Б-52', 'town': {...","{'team': {'id': 258, 'name': 'МордоBOYS', 'tow...","{'team': {'id': 558, 'name': 'НТР', 'town': {'...","{'team': {'id': 285, 'name': 'Лис', 'town': {'...","{'team': {'id': 361, 'name': '1Ф', 'town': {'i...",...,None,None,None,None,None,None,None,None,None,None


In [5]:
for i in range(5528):
    if results_df.iloc[i][0]['mask'] is not None:
        break
results_df.iloc[i][0]

{'team': {'id': 1, 'name': 'Неспроста', 'town': {'id': 201, 'name': 'Москва'}},
 'mask': '011101110110111000110111001111111111001111110001111111111111111111110101111100011000111111',
 'current': {'name': 'КП - Неспроста', 'town': {'id': 201, 'name': 'Москва'}},
 'questionsTotal': 67,
 'synchRequest': None,
 'position': 1,
 'controversials': [],
 'flags': [],
 'teamMembers': [{'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 1560,
    'name': 'Евгений',
    'patronymic': 'Наполеонович',
    'surname': 'Арутюнов'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 2935,
    'name': 'Анатолий',
    'patronymic': 'Рафаилович',
    'surname': 'Белкин'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 3270,
    'name': 'Юрий',
    'patronymic': 'Германович',
    'surname': 'Бершидский'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 4878,
    'name': 'Сергей',
    'patronymic': 'Владимирович',


## объединим

Из results можно узнать информацию по каждой команде, принимавшей участие в конкретном турнире, в частности составы команд и повопросные результаты данной команды

Индекс строки в results соответствует id из tournaments. Объединим их, оставив из tournaments_df только name и dateStart, и преобразуем дату к удобному формату

In [6]:
results_df['id'] = results_df.index
res_tour_df = tournaments_df[['id', 'name', 'dateStart']].merge(results_df, how="left", on="id")
res_tour_df['dateStart'] = pd.to_datetime(res_tour_df['dateStart'], utc = True)
res_tour_df.head(2)

,id,name,dateStart,0,1,2,3,4,5,6,...,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063
0,1,Чемпионат Южного Кавказа,2003-07-24 20:00:00+00:00,"{'team': {'id': 242, 'name': 'Команда Азимова'...","{'team': {'id': 640, 'name': 'Перезагрузка', '...","{'team': {'id': 245, 'name': 'Айастан', 'town'...","{'team': {'id': 299, 'name': 'Команда Гусейнов...","{'team': {'id': 1189, 'name': 'Грааль', 'town'...","{'team': {'id': 251, 'name': 'Пентиум', 'town'...","{'team': {'id': 268, 'name': 'Команда Алиева',...",...,None,None,None,None,None,None,None,None,None,None
1,2,Летние зори,2003-08-08 20:00:00+00:00,"{'team': {'id': 312, 'name': 'Социал-демократы...","{'team': {'id': 510, 'name': 'Импульс', 'town'...","{'team': {'id': 195, 'name': 'Реал-Мордовия', ...","{'team': {'id': 264, 'name': 'Джокер', 'town':...","{'team': {'id': 240, 'name': 'Бегемот', 'town'...","{'team': {'id': 689, 'name': 'Б-52', 'town': {...","{'team': {'id': 258, 'name': 'МордоBOYS', 'tow...",...,None,None,None,None,None,None,None,None,None,None


Теперь преобразуем данный датафрейм таким образом, чтобы в каждой строке была информация об ответах на вопросы конкретной команды на конкретном турнире

для каждого айди турнира, для каждого айди команды записать tour_id, tour_name, team_id, team_name, date_start, mask, team_members_ids, но если нет mask или teamMembers, то такую строку опускаем

In [7]:
data_for_model_list = []
counter_written = 0
counter_read = 0

for i in range(results_df.shape[0]):
    row = res_tour_df.iloc[i]
    tour_id = row['id']
    tour_name = row['name']
    date_start = row['dateStart']

    for j in range(results_df.shape[1] - 1):
        counter_read += 1
        
        team = row[j]
        if team is None:
            continue
            
        #team id and name
        if 'team' in team:
            #id
            if 'id' in team['team']:
                team_id = team['team']['id']
            else:
                team_id = None
            #name
            if 'name' in team['team']:
                team_name = team['team']['name']
            else:
                team_name = None
        else:
            team_id = None
            team_name = None
        
        #mask
        if "mask" in team:
            mask = team["mask"]
            if mask is None:
                continue
        else:
            continue
        
        #team_memders_ids
        if 'teamMembers' in team:
            team_members_ids = []
            for member in team['teamMembers']:
                if 'player' in member:
                    if 'id' in member['player']:
                        team_members_ids.append(member['player']['id'])
            if team_members_ids == []:
                continue
        else:
            continue
            
        data_for_model_list.append([
            tour_id, tour_name, team_id, team_name,
            date_start, mask, team_members_ids
        ])
        
        counter_written += 1
        if counter_written % 10000 == 0:
            print(f'{counter_written} rows written! {counter_read} elements read!')
        


10000 rows written! 2733547 elements read!
20000 rows written! 3118803 elements read!
30000 rows written! 3442837 elements read!
40000 rows written! 3680544 elements read!
50000 rows written! 4023290 elements read!
60000 rows written! 4186529 elements read!
70000 rows written! 4340606 elements read!
80000 rows written! 4540840 elements read!
90000 rows written! 4666851 elements read!
100000 rows written! 4833963 elements read!
110000 rows written! 5083634 elements read!
120000 rows written! 5298465 elements read!
130000 rows written! 5455215 elements read!
140000 rows written! 5560493 elements read!
150000 rows written! 5723823 elements read!
160000 rows written! 5985878 elements read!
170000 rows written! 6136703 elements read!
180000 rows written! 6318173 elements read!
190000 rows written! 6405039 elements read!
200000 rows written! 6627529 elements read!
210000 rows written! 6910349 elements read!
220000 rows written! 7129122 elements read!
230000 rows written! 7235419 elements rea

In [8]:
data_for_model = pd.DataFrame(
    data_for_model_list,
    columns = ['tour_id', 'tour_name', 'team_id', 'team_name', 'date_start', 'mask', 'team_members_ids']
)
data_for_model

,tour_id,tour_name,team_id,team_name,date_start,mask,team_members_ids
0,22,Чемпионат России,1,Неспроста,2004-02-20 21:00:00+00:00,0111011101101110001101110011111111110011111100...,"[1560, 2935, 3270, 4878, 18935, 32979, 36497]"
1,22,Чемпионат России,2,Афина,2004-02-20 21:00:00+00:00,0111111101011010010101110111111111110011011111...,"[707, 13551, 15442, 25882, 30475, 34846]"
2,22,Чемпионат России,670,Ксеп,2004-02-20 21:00:00+00:00,0011111101011010011101110011111111110111111111...,"[2694, 20691, 22482, 22935, 28513, 29800]"
3,22,Чемпионат России,173,ЮМА,2004-02-20 21:00:00+00:00,0111111001101110001101111011111111110100111111...,"[5526, 14259, 18686, 20260, 21805, 29333]"
4,22,Чемпионат России,188,Команда Губанова,2004-02-20 21:00:00+00:00,0111011001011111001101111011111101010111111101...,"[3207, 3645, 6212, 8333, 49151]"
...,...,...,...,...,...,...,...
439971,6456,Онлайн: 16:00 Зелёный шум,55612,Экстремаль,2020-05-02 13:00:00+00:00,100110001100111011011110100100010101010,"[138479, 80384, 133207, 95145, 121594, 133209]"
439972,6456,Онлайн: 16:00 Зелёный шум,68457,Орда,2020-05-02 13:00:00+00:00,100110101100001111110110100100010001010,[64129]
439973,6456,Онлайн: 16:00 Зелёный шум,43261,Господин Брейтенбихер,2020-05-02 13:00:00+00:00,000001111100111001010110000100111001011,"[111255, 166584, 135103, 103290, 103293]"
439974,6456,Онлайн: 16:00 Зелёный шум,69918,Teddyhaters,2020-05-02 13:00:00+00:00,100101101100100100010110100100010001010,"[96552, 192899, 192900, 123238, 129706, 192901]"


In [9]:
# data_for_model.to_csv('data_for_model.csv', index = False)

# Пункт 2: baseline-модель

*Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:*
- *повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;*
- *в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;*
- *для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.*


В предположении о том, что игрок ответил на вопрос тогда и только тогда, когда команда ответила на вопрос, можно рассмотреть следующую модель:
$$
p(x_{tj} = 1|s_i, c_j) \sim sigmoid(\mu + s_i + c_j), 
$$
где $x_{tj}$ - индикатор ответа $t$-й команды на $j$-ый вопрос, $s_i$ - сила $i$-го игрока, $c_j$ - сложность (а точнее простота) $j$-го вопроса 

Каждой команде на каждом турнире ставим в соответствие 0 или 1 в зависимости от того, играл ли конкретный игрок за эту команду или нет (каждая фича - индикатор конкретного игрока в команде, а выученный коэффициент при данном игроке - сила игрока), а также 0 или 1 в зависимости от того, был задан конкретный вопрос или нет (фичи - индикаторы вопроса, выученный коэффициент - простота вопроса)

Создадим общий айди для турнира и команды:

In [10]:
data_for_model['tour_team_id'] = (
    data_for_model['tour_id'].apply(str) 
    + '_' 
    + data_for_model['team_id'].apply(str)
)

Оставим только нужные маски (где только нули и/или единицы)

In [11]:
data_for_model['mask_flg'] = (
    (data_for_model['mask'].apply(set) == {'0', '1'}) | 
    (data_for_model['mask'].apply(set) == {'0'}) |
    (data_for_model['mask'].apply(set) == {'1'})
)
data_for_model = data_for_model[data_for_model['mask_flg'] == True]
len(data_for_model)

356254

Преобразуем поле mask так, чтобы оно было в виде списка

In [12]:
data_for_model['mask_list'] = data_for_model['mask'].apply(list)

/var/folders/cb/27mzjfxs377c11b1s98p_6y40000gn/T/ipykernel_1337/1375979770.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_for_model['mask_list'] = data_for_model['mask'].apply(list)


In [13]:
data_for_model.head(1)

,tour_id,tour_name,team_id,team_name,date_start,mask,team_members_ids,tour_team_id,mask_flg,mask_list
0,22,Чемпионат России,1,Неспроста,2004-02-20 21:00:00+00:00,0111011101101110001101110011111111110011111100...,"[1560, 2935, 3270, 4878, 18935, 32979, 36497]",22_1,True,"[0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, ..."


На каждом турнире одинаковые вопросы. Сопоставим каждому ответу из поля mask айди соответствующего вопроса при помощи tour_id + позиция вопроса

Сначала просто сопоставим номера вопросов.

In [14]:
data_for_model['questions_ids'] = data_for_model['mask_list'].apply(lambda x: [str(i) for i in range(len(x))])

data_for_model.head(2)


/var/folders/cb/27mzjfxs377c11b1s98p_6y40000gn/T/ipykernel_1337/3081311162.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_for_model['questions_ids'] = data_for_model['mask_list'].apply(lambda x: [str(i) for i in range(len(x))])


,tour_id,tour_name,team_id,team_name,date_start,mask,team_members_ids,tour_team_id,mask_flg,mask_list,questions_ids
0,22,Чемпионат России,1,Неспроста,2004-02-20 21:00:00+00:00,0111011101101110001101110011111111110011111100...,"[1560, 2935, 3270, 4878, 18935, 32979, 36497]",22_1,True,"[0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,22,Чемпионат России,2,Афина,2004-02-20 21:00:00+00:00,0111111101011010010101110111111111110011011111...,"[707, 13551, 15442, 25882, 30475, 34846]",22_2,True,"[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."


Теперь будем строить модель. Напишем функцию, которая вернет нам X и y (чтобы перейти от полей mask_list и questions_ids к  вышеописанным фичам, нужно будет применить функцию explode для mask_list и questions_ids, затем составить уникальные айди вопроса, прибавив к questions_ids tour_id, а затем при помощи функции get_dummies сделать one-hot кодирование для team_members_ids и questions_ids)

In [15]:
def get_matrix(data_for_model, year = '2019'):
    data_tmp = data_for_model.copy()
    data_tmp['date_start'] = data_tmp['date_start'].apply(lambda x: x.strftime('%Y'))
    data_tmp = data_tmp[data_tmp['date_start'] == year]
    
    data_tmp = data_tmp.explode('team_members_ids').explode(['mask_list', 'questions_ids'])
    data_tmp['questions_ids'] = (
        data_tmp['tour_id'].apply(str) + '_' + data_tmp['questions_ids'].apply(str)
    )
    
    print(f"Unique questions: {len(data_tmp['questions_ids'].unique())}")
    print(f"Unique members: {len(data_tmp['team_members_ids'].unique())}")
    
    data_tmp = data_tmp[['team_members_ids', 'questions_ids', 'mask_list']]
    data_tmp = pd.get_dummies(data_tmp, columns = ['team_members_ids'], sparse = True)
    data_tmp = pd.get_dummies(data_tmp, columns = ['questions_ids'], sparse = True)

    matrix = data_tmp.drop(columns=['mask_list']).sparse.to_coo()
    y = data_tmp['mask_list'].apply(int).to_numpy()
    columns = data_tmp.columns
    return matrix, y, columns

In [16]:
%%time
matrix_train, y_train, columns_train = get_matrix(data_for_model)

Unique questions: 30593
Unique members: 57465


/Users/romanzilotov/opt/anaconda3/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


CPU times: user 47.2 s, sys: 12 s, total: 59.2 s
Wall time: 1min 2s


In [17]:
matrix_train.shape

(17392830, 88058)

Обучим модель

In [25]:
%%time
baseline_model = LogisticRegression(max_iter = 100, solver = 'sag').fit(matrix_train, y_train)

CPU times: user 3min 34s, sys: 830 ms, total: 3min 34s
Wall time: 3min 34s


In [26]:
y_train_pred = baseline_model.predict(matrix_train)

print(f"Accuracy: {baseline_model.score(matrix_train, y_train)}")

Accuracy: 0.7651931859277645


In [27]:
pickle.dump(baseline_model, open('baseline_sag_100_2.pkl', 'wb'))

In [18]:
baseline_model = pickle.load(open('baseline_sag_100_2.pkl', 'rb'))

Первые num_members коэффициентов модели соответствуют рейтингам игроков. Сопоставим их

In [19]:
num_members = 57465

members_coefs = baseline_model.coef_[0][:num_members]
members_scores = 1 / (1 + np.exp(-members_coefs))
members_ids = [int(column[17:]) for column in columns_train[1:][:num_members]]

score_map = pd.DataFrame(np.array([members_ids, members_scores]).T, columns = ['id', 'score'])
score_map['id'] = score_map['id'].apply(int)

score_map = players_df.merge(score_map, how="inner", on="id").sort_values(by = 'score', ascending = False)
score_map.head(10)

,id,name,patronymic,surname,score
3804,27403,Максим,Михайлович,Руссо,0.982444
593,4270,Александра,Владимировна,Брутер,0.979933
3993,28751,Иван,Николаевич,Семушин,0.979340
3876,27822,Михаил,Владимирович,Савченков,0.976285
4178,30152,Артём,Сергеевич,Сорожкин,0.976216
4198,30270,Сергей,Леонидович,Спешков,0.976106
2531,18036,Михаил,Ильич,Левандовский,0.973384
9203,87637,Антон,Владимирович,Саксонов,0.972723
2884,20691,Станислав,Григорьевич,Мереминский,0.971857
3178,22799,Сергей,Игоревич,Николенко,0.971619


In [20]:
score_map.tail(10)

,id,name,patronymic,surname,score
42427,203840,Владислав,Вячеславович,Овчинников,0.018941
45920,207707,Антон,Денисович,Сычкин,0.018609
45917,207704,Анастасия,Михайловна,Ильина,0.018609
45919,207706,Иван,Владимирович,Насыров,0.018609
45916,207703,Дарья,Александровна,Петрушова,0.018609
45918,207705,Кристина,Александровна,Малыгина,0.018609
45915,207702,Дарья,Александровна,Безъязыкова,0.018609
42429,203842,Вероника,Андреевна,Балакина,0.018175
42432,203845,Андрей,Кириллович,Бурштын,0.017454
42430,203843,Анастасия,Павловна,Чурсина,0.016026


Можно сделать вывод, что модель работает разумно, так как в топе по рейтингу можно видеть известных и успешных игроков, а в хвосте -- людей, информацию по которым найти не получается.

# Пункт 3: предсказываем результаты турниров

*Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:*
- *предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;*
- *в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.*

Задача: на тестовой выборке отранжировать команды в каждом из проведенных турниров. В качестве ранжирующей метрики будем использовать силу команды

Силу команды будем определять как вероятность команды ответить на вопрос, что есть вероятность ответа на вопрос хотя бы одного из игроков. Это можно вычислить как 
$$
p_{team} = 1 - \prod_{i \in team}(1 - p_i),
$$
где $p_i$ - вычисленный ранее рейтинг для $i$-го игрока

Напишем функцию, которая по айди игрока вычисляет его рейтинг (для неизвестного ранее игрока будем выдавать рейтинг 0.5, что практически совпадает со средним и медианой)

In [22]:
def get_rating(member_id):
    if member_id in score_map['id'].to_numpy():
        index = score_map['id'][score_map['id'] == member_id].index.item()
        return score_map['score'][index]
    else:
        return 0.5

Выгрузим нужные данные для сравнения предсказанного рейтинга команды с реальными результатами.

In [23]:
data_for_team_rating_list = []
counter_written = 0
counter_read = 0

for i in range(results_df.shape[0]):
    row = res_tour_df.iloc[i]
    tour_id = row['id']
    tour_name = row['name']
    date_start = row['dateStart']

    for j in range(results_df.shape[1] - 1):
        counter_read += 1
        
        team = row[j]
        if team is None:
            continue
            
        #team id and name
        if 'team' in team:
            #id
            if 'id' in team['team']:
                team_id = team['team']['id']
            else:
                team_id = None
            #name
            if 'name' in team['team']:
                team_name = team['team']['name']
            else:
                team_name = None
        else:
            team_id = None
            team_name = None
        
        #mask
        if "mask" in team:
            mask = team["mask"]
            if mask is None:
                continue
        else:
            continue
        
        #team_memders_ids
        if 'teamMembers' in team:
            team_members_ids = []
            for member in team['teamMembers']:
                if 'player' in member:
                    if 'id' in member['player']:
                        team_members_ids.append(member['player']['id'])
            if team_members_ids == []:
                continue
        else:
            continue
            
        # j is position of the team on this tournament
        data_for_team_rating_list.append([
            tour_id, tour_name, team_id, team_name,
            date_start, mask, team_members_ids, j
        ])
        
        counter_written += 1
        if counter_written % 10000 == 0:
            print(f'{counter_written} rows written! {counter_read} elements read!')
        


10000 rows written! 2733547 elements read!
20000 rows written! 3118803 elements read!
30000 rows written! 3442837 elements read!
40000 rows written! 3680544 elements read!
50000 rows written! 4023290 elements read!
60000 rows written! 4186529 elements read!
70000 rows written! 4340606 elements read!
80000 rows written! 4540840 elements read!
90000 rows written! 4666851 elements read!
100000 rows written! 4833963 elements read!
110000 rows written! 5083634 elements read!
120000 rows written! 5298465 elements read!
130000 rows written! 5455215 elements read!
140000 rows written! 5560493 elements read!
150000 rows written! 5723823 elements read!
160000 rows written! 5985878 elements read!
170000 rows written! 6136703 elements read!
180000 rows written! 6318173 elements read!
190000 rows written! 6405039 elements read!
200000 rows written! 6627529 elements read!
210000 rows written! 6910349 elements read!
220000 rows written! 7129122 elements read!
230000 rows written! 7235419 elements rea

In [24]:
data_for_team_rating = pd.DataFrame(
    data_for_team_rating_list,
    columns = [
        'tour_id', 'tour_name', 'team_id', 'team_name', 
        'date_start', 'mask', 'team_members_ids', 'team_result'
    ]
)
data_for_team_rating.head(3)

,tour_id,tour_name,team_id,team_name,date_start,mask,team_members_ids,team_result
0,22,Чемпионат России,1,Неспроста,2004-02-20 21:00:00+00:00,0111011101101110001101110011111111110011111100...,"[1560, 2935, 3270, 4878, 18935, 32979, 36497]",0
1,22,Чемпионат России,2,Афина,2004-02-20 21:00:00+00:00,0111111101011010010101110111111111110011011111...,"[707, 13551, 15442, 25882, 30475, 34846]",1
2,22,Чемпионат России,670,Ксеп,2004-02-20 21:00:00+00:00,0011111101011010011101110011111111110111111111...,"[2694, 20691, 22482, 22935, 28513, 29800]",2


In [25]:
data_for_team_rating['date_start'] = data_for_team_rating['date_start'].apply(lambda x: x.strftime('%Y'))
data_for_team_rating = data_for_team_rating[data_for_team_rating['date_start'] == '2020']

data_for_team_rating['mask_flg'] = (
    (data_for_team_rating['mask'].apply(set) == {'0', '1'}) | 
    (data_for_team_rating['mask'].apply(set) == {'0'}) |
    (data_for_team_rating['mask'].apply(set) == {'1'})
)
data_for_team_rating = data_for_team_rating[data_for_team_rating['mask_flg'] == True]


data_for_team_rating.head(1)

,tour_id,tour_name,team_id,team_name,date_start,mask,team_members_ids,team_result,mask_flg
351083,5414,Синхрон северных стран,66120,Чатик,2020,111111111111111111111110111110101111,"[18490, 116901, 8532, 42346, 123190, 22482]",0,True


Вычислим рейтинг команды

In [27]:
def get_team_rating(team_members_ids):
    team_rating = 1
    for member_id in team_members_ids:
        team_rating *= 1 - get_rating(member_id)
    return 1 - team_rating

In [28]:
data_for_team_rating['team_rating'] = data_for_team_rating['team_members_ids'].apply(get_team_rating)

In [29]:
data_for_team_rating.head(2)

,tour_id,tour_name,team_id,team_name,date_start,mask,team_members_ids,team_result,mask_flg,team_rating
351083,5414,Синхрон северных стран,66120,Чатик,2020,111111111111111111111110111110101111,"[18490, 116901, 8532, 42346, 123190, 22482]",0,True,1.0
351084,5414,Синхрон северных стран,312,Социал-демократы,2020,111111110111111111111011101010111111,"[19599, 3577, 15051, 25724, 34508, 61400]",1,True,1.0


In [30]:
data_for_team_rating['team_rating'].sort_values()

420059    0.032732
420628    0.045661
383667    0.045720
420037    0.054534
420638    0.054788
            ...   
439083    1.000000
437872    1.000000
437876    1.000000
437864    1.000000
437860    1.000000
Name: team_rating, Length: 19264, dtype: float64

In [31]:
data_for_team_rating['mean_answered'] = data_for_team_rating['mask'].apply(
    lambda x: np.array(list(x)).astype(int).mean()
)
data_for_team_rating.head(1)

,tour_id,tour_name,team_id,team_name,date_start,mask,team_members_ids,team_result,mask_flg,team_rating,mean_answered
351083,5414,Синхрон северных стран,66120,Чатик,2020,111111111111111111111110111110101111,"[18490, 116901, 8532, 42346, 123190, 22482]",0,True,1.0,0.916667


Итого:
- team_result - результат команды на турнире
- mean_answered - частота правильных ответов
- team_rating - вычисленный рейтинг команды

Выведем теперь искомые метрики для каждой пары

In [32]:
def get_corr(data, col_1, col_2, metric_name):
    if metric_name == 'spearman':
        return spearmanr(data[col_1], data[col_2]).correlation
    if metric_name == 'kendall':
        return kendalltau(data[col_1], data[col_2]).correlation


In [33]:
for cols in combinations(['team_result', 'team_rating', 'mean_answered'], 2):
    print(f'{cols[0]} vs {cols[1]}:')
    for metric in ['spearman', 'kendall']:
        print(f'\t{metric} corr: {data_for_team_rating.groupby("tour_id").apply(get_corr, cols[0], cols[1], metric).mean()}')
        

team_result vs team_rating:
	spearman corr: -0.7548112909463381
	kendall corr: -0.5862575132112186
team_result vs mean_answered:
	spearman corr: -0.9938080397909352
	kendall corr: -0.971241499956389
team_rating vs mean_answered:
	spearman corr: 0.7535762396779663
	kendall corr: 0.5959062567153657


Можно видеть, что среднее число ответов и позиция команды в итоговом протоколе практически одинаковы в плане корреляции (только имеют знак минус из-за обратной пропорциональности), а метрики относительно предсказанного рейтинга находятся в пределах адекватности